In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
import random as rd
from sklearn.decomposition import PCA
from IPython.display import display
import os 
from pathlib import Path
import shutil 
import cv2

In [2]:
### Vamos a sacar los  17 primeros landmarks--lo  hacemos de la frame 0  de cada secuencia
##Por ahora acá no voy a usarlas alineadas<---PREGUNTAR

In [3]:
subjects_landmarks = '../ck+copy/Landmarks/'
sorted_subjects =   np.sort(os.listdir('../ck+copy/Landmarks/')) # we probably need to not have the sorted

distances = []
for subject in sorted_subjects:
    one_subject_landmarks = '../ck+copy/Landmarks/' + str(subject)+ '/'
    sorted_list = np.sort(os.listdir(one_subject_landmarks)) #creo que este sort se puede quitar
    distances_per_subject = []
    for emotion in sorted_list:
        matrix_landmark_per_frame = []
        subject_landmark_emotion = one_subject_landmarks + str(emotion)    
        for i in range(0, len(os.listdir(subject_landmark_emotion))):
            data = pd.read_csv('../ck+copy/Landmarks/' +  str(subject)+ '/' + str(emotion)+'/'+ str(subject)+ '_'+str(emotion)+'_'+ str(i+1).zfill(8)+'_landmarks.txt',  header=None)
            data.columns =['landmarks']
            quantity_of_landmarks = data.size #68
            data = data['landmarks'].str.split('   ', expand=True)
            data.columns =['landmarks', 'landmarkX', 'landmarkY']
            data =data.drop(columns=['landmarks'])
            dataarray = data.to_numpy(dtype=float)
            matrix_landmark_per_frame.append(dataarray)
        
        distances_per_subject.append(matrix_landmark_per_frame)
    distances.append(distances_per_subject)   

In [4]:
for i in range(0, len(distances)):
    for j in range(0, len( distances[i])):
        distances[i][j]= np.asarray( distances[i][j], dtype=object)
    distances[i]= np.asarray( distances[i], dtype=object)
distances= np.asarray( distances, dtype=object)


In [5]:
len(distances[0][0])

29

In [6]:
first_frame =[]
cant_frame_per_seq = []
for i in range(0, len(distances)):
    for j in range(0, len( distances[i])):
        first_frame.append(distances[i][j][0])
        cant_frame_per_seq.append(len(distances[i][j]))
        

first_frame = np.asarray( first_frame, dtype=object)
cant_frame_per_seq = np.asarray( cant_frame_per_seq, dtype=object)

In [7]:
first_frame.shape

(51, 68, 2)

In [8]:
cant_frame_per_seq

array([29, 16, 30, 19, 17, 30, 17, 33, 23, 32, 17, 19, 12, 29, 25, 14, 24,
       15, 14, 10, 21, 13, 29, 28, 14, 21, 14, 33, 17, 19, 27, 22, 20, 13,
       14, 15, 13, 5, 23, 16, 14, 15, 7, 6, 27, 20, 10, 22, 9, 15, 18],
      dtype=object)

In [9]:
first_frame[0]

array([[245.688, 252.45154],
       [250.75381, 282.80546],
       [259.36702, 312.51759],
       [266.68465, 342.4186],
       [277.26378, 371.34904],
       [295.88027, 395.58941],
       [319.49635, 415.18508],
       [346.07042, 431.09897],
       [375.99423, 434.84148],
       [406.69956, 428.90793],
       [433.89479, 413.33251],
       [456.79126, 392.04819],
       [474.23993, 366.08947],
       [482.94386, 335.99443],
       [488.31702, 305.20952],
       [493.74997, 274.06672],
       [496.64832, 243.13984],
       [265.41412, 215.47898],
       [283.89499, 205.55665],
       [304.25895, 201.28303],
       [324.85539, 202.94621],
       [344.74546, 208.83489],
       [393.71675, 206.22633],
       [413.20441, 198.00231],
       [433.98556, 194.42791],
       [454.67283, 197.57962],
       [473.53553, 206.85074],
       [369.43201, 238.88037],
       [369.70983, 258.08105],
       [370.28162, 277.01609],
       [370.77023, 295.95668],
       [348.72465, 320.63784],
       [360

In [11]:
#por ahora usamos la distancia horizontal (ver 1)

In [10]:
#usamos 19 y 6 porque son los mas cercanos a nivel de temple de los landmarks de ck+ y asumimos hay correlacion entre la distancia de estos y la distancia que tendria gente con temples mas largos
#revisar que la asumpsion sea verdad
#Acordemonos que los arry el indice va de 0-67 y en iomagenes de 1 a 68, en codigo usamos -1 del que se usa en la imagen

first_feature_point = []
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][18][0]
    point2 = first_frame[i][25][0]
    first_feature_point.append(np.abs(point2-point1))
first_feature_point = np.asarray( first_feature_point, dtype=object)
first_feature_point
#19,26

array([170.77783999999997, 168.93462, 170.06516999999997, 171.19804,
       168.32029, 129.55662999999998, 131.46166999999997, 127.38072,
       130.84753999999998, 130.84129000000001, 128.40718999999996,
       131.88854000000003, 147.9945, 151.98522, 154.56719999999999,
       155.45786, 156.06501, 157.25549, 153.98916999999997,
       156.65437000000003, 142.65292, 167.49652999999998, 162.5411,
       163.45542, 159.85144999999997, 160.87183000000002,
       162.35458999999997, 148.51279, 151.28485999999998,
       147.49376999999998, 145.65249, 152.04866, 149.8119,
       152.02220000000003, 169.27717, 166.48325999999997, 169.64881,
       167.52213, 167.15892000000002, 170.28283, 177.62755, 180.96388,
       180.93162, 181.47614000000002, 181.48164, 179.35411,
       156.18697999999998, 205.67178, 209.61372000000003,
       209.09468999999999, 204.73733], dtype=object)

In [11]:
second_feature_point = []
for i in range(0, len(first_frame)):
    point1 = first_frame[i][1][0]
    point2 = first_frame[i][15][0]
    second_feature_point.append(np.abs(point2-point1))
second_feature_point = np.asarray( second_feature_point, dtype=object)
second_feature_point
#2,16

array([242.99616000000003, 244.42923000000002, 244.29061000000002,
       246.06672, 245.47744, 197.57958000000002, 196.48839999999998,
       194.02043, 196.54987000000003, 196.31359999999998, 195.4785,
       194.13973, 224.02455, 218.59473999999997, 224.36149000000003,
       224.19346999999996, 216.88727000000003, 217.59705999999997,
       214.58100000000002, 213.60518999999996, 213.61859,
       221.32809999999998, 218.55567, 219.08137000000002,
       218.20252000000002, 221.31825000000003, 218.63573, 221.25832,
       223.21594000000002, 223.98555, 222.09176999999997,
       222.69307999999998, 223.71244, 225.8115, 244.74725, 244.19702,
       244.19439000000003, 241.97357999999997, 243.73223000000002,
       244.77529, 242.72324000000003, 245.15591999999998, 244.79638,
       244.94695000000002, 241.44837000000004, 243.06213999999997,
       219.76012000000003, 273.50397999999996, 271.40627,
       272.76027999999997, 271.93015], dtype=object)

In [12]:
third_feature_point = []
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][1][0]
    point2 = first_frame[i][14][0]
    third_feature_point.append(np.abs(point2-point1))
third_feature_point = np.asarray( third_feature_point, dtype=object)
third_feature_point
#3,15

array([237.56321000000003, 239.14769, 237.73748999999998,
       240.14542000000003, 239.78365000000002, 197.45231,
       195.01178000000002, 190.97196, 194.52761, 193.89962999999997,
       192.31806999999998, 191.44419, 220.09828000000002, 215.56179,
       218.90326000000002, 216.33124999999998, 214.60825000000003,
       215.42241999999996, 215.32493, 213.94082999999998, 211.96496,
       220.59079, 215.89115999999999, 216.79227999999998,
       214.74267999999998, 218.78919000000002, 215.9421, 214.54554,
       216.28195000000002, 219.01794999999998, 216.19364,
       215.67666999999997, 217.63358000000002, 220.84263, 237.68464,
       234.8775, 238.53817, 236.08321, 236.86639000000002, 238.24765,
       238.37414, 240.87176, 239.5765, 243.10806000000002,
       236.39781000000002, 236.82104999999999, 213.29798, 272.10934,
       270.46026, 269.88485, 270.94245], dtype=object)

In [14]:
#asumimos la mean horizontalmente va a dar la distancia horizontal del punto intermedio
fourth_feature_point = []
for i in range(0, len(first_frame)):   
    point1a = first_frame[i][3][0]
    point2a = first_frame[i][13][0]        
    point1b = first_frame[i][4][0]
    point2b = first_frame[i][12][0]
    point1 = (point1a + point1b)/2#mean
    point2 = (point2a + point2b)/2
    fourth_feature_point.append(np.abs(point2-point1))
fourth_feature_point = np.asarray( fourth_feature_point, dtype=object)
fourth_feature_point
#diferencia de mean entre 4 y 5, 14 y 13,  

array([206.61768000000006, 208.826775, 210.33306, 210.45923000000005,
       210.61737500000004, 168.567045, 165.43779, 165.59345999999994,
       168.685025, 168.37056999999993, 170.42478500000004, 167.048335,
       191.66966000000002, 192.3735, 192.05901, 192.729605, 198.246915,
       196.66978499999996, 195.48874, 195.04576, 199.00933999999998,
       194.319395, 190.98964, 191.53355499999998, 192.69318, 192.34116,
       190.37552999999997, 188.8671, 189.58366, 189.13129999999995,
       188.21261500000003, 187.28498000000002, 187.07253999999995,
       190.773905, 211.3568, 208.03021499999997, 210.13388500000002,
       208.64860499999998, 209.185365, 210.94231500000004,
       209.63739999999996, 212.22577500000006, 213.41444999999996,
       212.31738, 211.73346500000002, 213.18663999999995,
       191.36537500000003, 251.71267999999998, 250.18885500000005,
       249.42301000000003, 251.81550500000003], dtype=object)

In [15]:
five_feature_point = []
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][6][0]
    point2 = first_frame[i][10][0]
    five_feature_point.append(np.abs(point2-point1))
five_feature_point = np.asarray( five_feature_point, dtype=object)
five_feature_point
#7,11

array([114.39844, 117.83312000000001, 116.06396000000001,
       116.55308000000002, 115.72111000000001, 92.34618,
       95.12004999999999, 91.22553999999997, 96.37851999999998, 92.81734,
       98.12718999999998, 92.99597, 109.06219000000002, 111.37859,
       112.62211000000002, 110.90966000000003, 109.18882000000002,
       106.38199, 106.98391000000001, 106.12100000000001, 108.81187,
       103.28314999999998, 109.41278000000003, 107.71508999999998,
       106.08174000000002, 108.81124000000003, 105.79146000000003,
       105.82914, 106.36009000000001, 105.13608000000005,
       107.19103000000001, 106.32248999999999, 109.20044000000001,
       110.94890000000001, 112.97381000000001, 112.35740999999999,
       113.02426, 113.04085000000003, 111.91798, 111.64093,
       118.15116999999998, 118.25062999999997, 120.08277000000001,
       121.39840000000001, 119.12638999999999, 117.39411999999999,
       109.78521999999998, 133.13085, 131.68364000000003, 132.91104,
       136.72561], 

In [16]:
six_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][31][0]
    point2 = first_frame[i][35][0]
    six_feature_point.append(np.abs(point2-point1))
six_feature_point = np.asarray( six_feature_point, dtype=object)
six_feature_point
#32,36

array([46.002459999999985, 47.573459999999955, 50.37164999999999,
       48.91736000000003, 47.700710000000015, 36.29955000000001,
       33.960890000000006, 35.974789999999985, 38.51358000000005,
       35.275769999999966, 37.67982000000001, 47.29204999999996,
       44.03095999999999, 42.380420000000015, 39.40820000000002,
       44.05560000000003, 52.62810000000002, 49.441689999999966,
       43.70517000000001, 42.25925999999998, 53.80801999999997,
       48.90110000000004, 44.22910999999999, 44.468129999999974,
       45.70139999999998, 43.54512, 42.06837999999999, 40.40829000000002,
       44.444339999999954, 38.32390000000004, 40.52332000000001,
       37.60826000000003, 39.715400000000045, 41.64678999999995,
       39.16867000000002, 37.12783000000002, 38.561019999999985,
       36.21756999999997, 37.12158999999997, 39.239480000000015,
       69.62619999999998, 66.22681, 67.06600000000003, 65.57173,
       67.30680000000001, 67.60883000000001, 41.41097000000002,
       64.125369

In [17]:
seven_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][61][0]
    point2 = first_frame[i][65][0]
    seven_feature_point.append(np.abs(point2-point1))
seven_feature_point = np.asarray( seven_feature_point, dtype=object)
seven_feature_point
#61,65

array([24.649040000000014, 29.27212000000003, 30.33069999999998,
       29.592300000000023, 28.068979999999954, 21.404629999999997,
       23.045680000000004, 23.574039999999968, 23.31711999999999,
       22.358220000000017, 22.10163, 36.93379999999996, 22.83148,
       23.09147999999999, 22.345950000000016, 41.54285999999996,
       25.704489999999964, 28.11425, 41.76231999999999, 42.36403999999999,
       45.60861999999997, 34.050509999999974, 35.04303999999996,
       36.72737000000001, 31.82704000000001, 33.23340999999999,
       33.54767000000004, 20.330319999999972, 21.923249999999996,
       23.46078, 23.266509999999982, 22.797169999999994,
       21.665020000000027, 39.84535999999997, 31.01967000000002,
       30.557979999999986, 29.26596999999998, 31.101459999999975,
       29.747210000000052, 28.056579999999997, 39.563890000000015,
       40.56035000000003, 42.18499000000003, 40.23010999999997,
       49.30556999999999, 50.959, 42.33797000000004, 58.353080000000034,
       62

In [18]:
#asumimos la mean horizontalmente va a dar la distancia horizontal del punto intermedio
eight_feature_point = []
for i in range(0, len(first_frame)):   
    point1a = first_frame[i][37][0]
    point2a = first_frame[i][43][0]        
    point1b = first_frame[i][38][0]
    point2b = first_frame[i][44][0]
    point1 = (point1a + point1b)/2#mean
    point2 = (point2a + point2b)/2
    eight_feature_point.append(np.abs(point2-point1))
eight_feature_point = np.asarray( eight_feature_point, dtype=object)
eight_feature_point
#dieferencia entre mean de 38-39 y 44-45

array([119.18116500000002, 116.83295999999996, 118.35316,
       118.02394499999991, 118.15443000000005, 86.32914,
       87.27517499999999, 86.58643999999998, 82.83473499999997, 85.08625,
       84.27823999999998, 93.07187500000003, 93.79263500000002, 95.786145,
       97.56446999999997, 92.59136000000001, 97.13424499999996,
       98.32676500000002, 95.56271000000004, 95.74266, 97.29689499999998,
       108.70876999999999, 110.82799499999999, 110.31047999999998,
       109.70598000000001, 112.01627000000002, 109.76085499999999,
       89.99258000000003, 89.95303000000001, 90.2595, 90.08671499999997,
       91.59902499999998, 91.10108499999996, 91.31486500000005,
       109.13162000000003, 111.47793000000001, 111.56210499999997,
       110.43670499999999, 110.72919000000002, 109.931895,
       102.72643000000005, 103.22321499999998, 101.43882000000002,
       101.964745, 103.63901000000004, 103.56627500000002,
       99.55260999999996, 125.67472500000002, 124.70084500000002,
       12

In [19]:
nine_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][42][0]
    point2 = first_frame[i][45][0]
    nine_feature_point.append(np.abs(point2-point1))
nine_feature_point = np.asarray( nine_feature_point, dtype=object)
nine_feature_point
#43 46

array([51.41291000000001, 51.15017, 56.84996000000001, 55.691340000000025,
       54.559860000000015, 35.001149999999996, 38.44709,
       36.228290000000015, 33.60449, 35.77386999999999, 40.50103999999999,
       33.82542999999998, 40.15244000000001, 39.79626000000002,
       40.57670999999999, 44.79680000000002, 37.53293000000002,
       39.24250000000001, 44.498339999999985, 44.356539999999995,
       46.07274000000001, 38.57808, 38.30173000000002, 39.167250000000024,
       38.597170000000006, 40.32446999999996, 38.25757999999996,
       40.00405000000001, 41.5634, 40.86660999999998, 39.909940000000006,
       42.19081, 42.30257999999998, 46.76688999999999, 43.292640000000006,
       45.054550000000006, 43.351699999999994, 45.634979999999985,
       45.36531000000002, 41.439059999999984, 43.803589999999986,
       45.20595000000003, 44.18829999999997, 43.443370000000016,
       50.43281999999999, 50.703449999999975, 46.17833999999999,
       59.16003999999998, 56.91854000000001, 57

In [20]:
ten_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][39][0]
    point2 = first_frame[i][42][0]
    ten_feature_point.append(np.abs(point2-point1))
ten_feature_point = np.asarray( ten_feature_point, dtype=object)
ten_feature_point
#40-43

array([69.86000999999999, 64.64231000000001, 63.32152000000002,
       63.650649999999985, 64.66537, 49.67561999999998, 49.07823000000002,
       50.58783, 50.21794999999997, 49.754440000000045, 47.9898,
       59.037409999999966, 55.27931000000001, 57.783180000000016,
       56.723040000000026, 48.19219999999996, 57.358239999999995,
       57.68609999999998, 51.66739000000001, 50.51757999999998,
       52.17429000000001, 68.63281, 70.98340000000002, 68.91091,
       67.76471999999995, 68.59812, 68.61796000000004, 49.518759999999986,
       48.29227000000003, 49.71836999999999, 48.20049999999998, 50.05507,
       52.24979999999999, 44.811019999999985, 67.73109, 68.61674,
       69.44612999999998, 65.95105000000001, 67.05129, 68.04049000000003,
       61.04767000000004, 60.35156999999998, 61.15163000000001,
       60.45715000000001, 56.67595, 56.12189999999998, 55.46085999999997,
       66.43509, 65.67982, 68.80474999999996, 83.29998], dtype=object)

In [22]:
eleven_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][36][0]
    point2 = first_frame[i][39][0]
    eleven_feature_point.append(np.abs(point2-point1))
eleven_feature_point = np.asarray( eleven_feature_point, dtype=object)
eleven_feature_point
#37 40

array([47.80718999999999, 55.123729999999966, 55.13198999999997,
       54.09541999999999, 52.140480000000025, 40.675070000000005,
       39.26644999999996, 36.97985, 32.09397000000001, 34.6927,
       34.36339000000004, 35.21338000000003, 40.58403999999999,
       39.37221999999997, 43.46955, 47.43306000000001, 40.49709999999999,
       40.48258000000001, 44.838959999999986, 46.557890000000015,
       43.28882999999999, 39.46400999999997, 38.29972999999998,
       39.167359999999974, 41.248020000000025, 42.670590000000004,
       40.338179999999994, 42.886089999999996, 42.83466999999999,
       43.406620000000004, 42.818460000000016, 44.13716999999997,
       41.88017000000002, 47.753690000000034, 40.34245999999999,
       41.51903999999999, 42.590750000000014, 44.16561999999999,
       42.51775999999998, 43.60672999999997, 45.574059999999974,
       46.03012000000001, 44.97759000000002, 46.91054, 49.93450999999999,
       50.06148000000002, 44.85014000000001, 58.50622000000001,
     

In [19]:
#estos son vertical distance

In [23]:
twelve_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][27][1]
    point2 = first_frame[i][33][1]
    twelve_feature_point.append(np.abs(point2-point1))
twelve_feature_point = np.asarray( twelve_feature_point, dtype=object)
twelve_feature_point
#28-34

array([85.19286000000002, 85.26593000000003, 82.72518, 84.73577,
       81.64027000000002, 72.69696000000002, 71.16573, 74.79878000000002,
       71.00049999999999, 73.34162, 62.96787999999998, 54.3364,
       79.87141999999997, 79.20645999999999, 78.88707, 86.58848,
       70.77699000000001, 69.61424000000002, 70.84064000000001, 71.9794,
       70.059, 83.65520999999998, 80.69811000000001, 88.78900999999999,
       87.21096999999997, 88.60242, 86.30506000000003, 68.08273,
       66.90150999999997, 67.40003999999999, 67.93413000000001,
       68.17873000000003, 71.66983000000002, 79.60194000000004,
       78.92867000000001, 79.76671000000002, 83.87159, 78.92781,
       73.78066999999999, 87.44486, 76.38666999999998, 78.77701000000002,
       78.39039, 79.47626, 83.86206999999999, 83.86332999999996, 87.249,
       101.44395999999998, 105.55780999999999, 102.29326999999998,
       102.73244], dtype=object)

In [24]:
thirteen_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][33][1]
    point2 = first_frame[i][8][1]
    thirteen_feature_point.append(np.abs(point2-point1))
thirteen_feature_point = np.asarray( thirteen_feature_point, dtype=object)
thirteen_feature_point
#34 9

array([110.76824999999997, 100.3175, 103.39454999999998,
       107.00492000000003, 106.29678999999999, 102.97696000000002,
       105.94421999999997, 102.71310999999997, 104.5736,
       104.14107000000001, 106.30690000000004, 91.00027999999998,
       117.37940000000003, 117.72199, 120.28562, 111.80210999999997,
       104.81504999999999, 102.83697999999998, 104.80451, 108.62939,
       100.10750000000002, 127.48154, 129.48134999999996, 130.31236,
       124.03861, 129.55374, 126.03174999999999, 94.36556000000002,
       102.70691000000005, 104.88641999999999, 101.88626,
       101.45824999999996, 100.61406, 99.26884999999999,
       113.07557999999997, 115.14842000000002, 113.05331000000001,
       116.35392000000002, 116.39790000000002, 117.09838999999997,
       116.21154999999999, 116.90812999999997, 115.32841000000002,
       118.05580000000003, 113.60248000000001, 118.92256000000003,
       112.54378999999997, 148.20011, 145.43834000000004,
       146.34543000000002, 145.154960

In [25]:
#asumimos la mean horizontalmente va a dar la distancia horizontal del punto intermedio
fourteen_feature_point = []
for i in range(0, len(first_frame)):   
    point1a = first_frame[i][38][1]
    point2a = first_frame[i][20][1]        
    point1b = first_frame[i][43][1]
    point2b = first_frame[i][23][1]
    point1 = (point1a + point1b)/2#mean
    point2 = (point2a + point2b)/2
    fourteen_feature_point.append(np.abs(point2-point1))
fourteen_feature_point = np.asarray( fourteen_feature_point, dtype=object)
fourteen_feature_point
#distancia de ños means de39-44 y 21-24


array([25.43394999999998, 31.849395000000015, 33.787980000000005,
       33.35919000000001, 31.797134999999997, 22.386184999999983,
       24.508229999999998, 23.896424999999994, 24.351009999999974,
       24.007569999999987, 23.472489999999993, 26.858494999999976,
       22.329559999999987, 22.5779, 23.319080000000014,
       24.749839999999978, 31.628094999999973, 31.23707999999999,
       28.89399499999996, 30.13246000000001, 24.87205499999999,
       34.96621999999999, 31.77130999999997, 32.73441500000001, 33.3348,
       32.97154500000002, 32.13674500000002, 24.42233999999999,
       24.553499999999985, 24.182995000000005, 25.711234999999988,
       26.034215000000017, 25.085139999999996, 26.187534999999997,
       32.040880000000016, 30.378005, 31.320525000000004,
       31.897154999999998, 29.48604500000002, 32.06499500000001,
       32.641549999999995, 36.535225, 37.858589999999964,
       37.578389999999985, 32.17262499999998, 34.141744999999986,
       26.559444999999982, 48.

In [26]:
#it says distance, I asumme is vertical pnly, but could be actual difference in witch case we use np.align
#creo que solo la vertical tiene + sentido para el height
fifteen_feature_point = []
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][37][1]
    point2 = first_frame[i][41][1]        
    point3 = first_frame[i][38][1]
    point4 = first_frame[i][40][1]     
    point5 = first_frame[i][43][1]
    point6 = first_frame[i][47][1]        
    point7 = first_frame[i][44][1]
    point8 = first_frame[i][46][1]
    point_pair1 = np.abs(point2-point1)
    point_pair2 = np.abs(point4-point3)
    point_pair3 = np.abs(point6-point5)
    point_pair4 = np.abs(point8-point7)
    fifteen_point = np.mean(np.array([point_pair1, point_pair2, point_pair3, point_pair4]))
    fifteen_feature_point.append(fifteen_point)
fifteen_feature_point = np.asarray( fifteen_feature_point, dtype=object)
fifteen_feature_point
#mean de las distancias de par 38-42, 39-41, 44-48, 45-47

array([17.363457500000003, 15.06223, 13.163489999999996,
       15.416887500000001, 13.8038125, 14.812505000000002,
       13.569245000000002, 13.744282500000004, 12.618985000000002,
       13.345924999999987, 14.52060500000001, 3.844930000000005,
       15.723605, 16.177172500000005, 16.210892499999993, 12.6752975,
       13.55037999999999, 13.408520000000003, 12.906537500000006,
       13.005870000000002, 11.276705000000007, 13.656700000000008,
       14.3531075, 15.359937499999994, 15.186212500000003,
       13.55447000000001, 13.22249, 13.629129999999996,
       13.108332499999996, 13.964640000000003, 13.393919999999994,
       15.004925, 14.359575, 13.3896625, 17.1515025, 17.325792500000006,
       18.1352125, 17.589789999999994, 16.636069999999997,
       17.158779999999993, 14.893019999999993, 14.333462500000003,
       14.270400000000002, 14.7078025, 13.669012500000001,
       13.478895000000001, 16.8914075, 14.520387499999998,
       13.904597500000001, 15.042067500000002, 6.9

In [27]:
sixteen_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][33][1]
    point2 = first_frame[i][56][1]
    sixteen_feature_point.append(np.abs(point2-point1))
sixteen_feature_point = np.asarray( sixteen_feature_point, dtype=object)
sixteen_feature_point
#34-57

array([48.99633, 49.86543999999998, 50.08084000000002, 53.78095000000002,
       49.25133999999997, 48.42476999999997, 48.52463, 45.52366000000001,
       50.73687000000001, 50.624059999999986, 51.98631, 40.02122000000003,
       50.85893000000004, 51.78203000000002, 53.379399999999976, 48.21499,
       41.43878000000001, 42.37847999999997, 43.292340000000024,
       44.75483000000003, 41.951329999999984, 55.91941000000003,
       54.50265999999999, 59.01902000000001, 55.004110000000026,
       57.01080999999999, 53.95152999999999, 44.09562999999997,
       43.026070000000004, 43.95587999999998, 43.90263999999996,
       42.03710000000001, 46.45740999999998, 40.039289999999994,
       51.791829999999976, 52.936949999999996, 51.704740000000015,
       54.13814000000002, 54.18267, 52.93573999999998, 58.11385000000001,
       57.41740999999996, 55.341920000000016, 58.05344000000002,
       59.58931000000001, 58.21849000000003, 49.88230999999999,
       76.50444000000005, 67.62918000000002

In [28]:
#no tenemos chin, usamos desde la boca hasta el fondo
seventeen_feature_point =[]
for i in range(0, len(first_frame)):   
    point1 = first_frame[i][67][1]
    point2 = first_frame[i][8][1]
    seventeen_feature_point.append(np.abs(point2-point1))
seventeen_feature_point = np.asarray( seventeen_feature_point, dtype=object)
seventeen_feature_point

#68-9

#HAY QUE CAMBIAR TODOS LOS INDICES

array([74.34746999999999, 65.12040000000002, 68.08348000000001,
       71.60160000000002, 69.47980999999999, 65.96454, 67.45283,
       66.62212999999997, 65.75743999999997, 65.49421000000001, 66.80291,
       62.821619999999996, 79.93736000000001, 79.59676999999999,
       80.10529000000002, 79.11417999999998, 76.11461000000003,
       73.51929999999999, 76.51299999999998, 75.54428000000001,
       73.28885000000002, 88.69857999999999, 90.61431999999996,
       89.66271999999998, 86.07155999999998, 91.77510999999998,
       89.76679999999999, 58.99261000000001, 67.79843, 68.25948,
       66.14613000000003, 69.03927999999996, 64.86788999999999,
       67.44631000000004, 78.91685999999999, 79.00750000000005,
       78.52897000000002, 79.21875, 80.29554000000002, 81.6651,
       78.59447999999998, 79.58364999999998, 80.93040000000002,
       80.93030000000005, 74.97501, 82.29521, 78.13221999999996,
       97.31737999999996, 101.17429000000004, 93.23665, 94.55027999999999],
      dtype=ob

In [26]:
#Ahora vienen los feature points que involucran shape y textura

In [29]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import numpy as np
eigtheen_feature_point = []
for i in range(0, len(first_frame)):   
    points = first_frame[i][39:59]
    hull = ConvexHull(points)
    eigtheen_feature_point.append(hull.volume)#cause is a 2d array volume == area
eigtheen_feature_point = np.asarray( eigtheen_feature_point, dtype=object)
eigtheen_feature_point

#40 a 60

array([16929.688541697597, 16825.895329713, 16951.82625665295,
       17249.8376208582, 16820.517535870396, 10692.643661647602,
       10610.107474816901, 10523.635431144847, 10515.7463111047,
       10496.235564302002, 10701.921786340952, 8520.42366207225,
       13182.2552882128, 13415.4689510437, 13323.158128681096,
       13430.240439551146, 11295.264277317749, 11581.671528186398,
       11294.756084516748, 11736.719252832145, 11849.798670261402,
       14840.617331517496, 15098.856033622757, 15082.470357333,
       14220.186879104298, 15077.449273355698, 14522.581718650998,
       11130.071558831296, 11057.836023950753, 11100.216072182997,
       10920.174774150653, 11396.2664522105, 11477.832497169404,
       11350.014115382646, 13280.506987798391, 13115.425615025846,
       13472.43065867635, 13329.8603109792, 13354.681553797504,
       13018.019238007295, 15580.271613836749, 15126.301237277403,
       15123.946751250054, 15333.596242563108, 16170.473512675004,
       15917.5892

In [ ]:
nineteen_feature_point<- Deberia ser average curvatura de cheeks

twenty_feature_point <- Deberia ser x co-ordinate of center of mass where mass rep-
resents curvature coefficient of cheeks area

twentyone_feature_point <- Deberia ser y co-ordinate of center of mass where mass rep-
resents curvature coefficient of cheeks area

### Vamos a empezar denuevo

### 

In [52]:
first_frame[0][14]

array([488.31702, 305.20952], dtype=object)

In [36]:
#VA A HABER QUE RASTERIZAR TAL VEZ.POR AHORA UN RECTANGULO COMUN
import numpy as np
'''
SHAPES PARA MAS ADELANTE
left_cheek = []
left_cheek.append(first_frame[0][1]
)
left_cheek.append(first_frame[0][39]
)
left_cheek.append(first_frame[0][28]
)
left_cheek.append(first_frame[0][48]
)
left_cheek.append(first_frame[0][4]
)
left_cheek.append(first_frame[0][3]
)
left_cheek.append(first_frame[0][2]
)
left_cheek = np.asarray(left_cheek)
print(left_cheek)
#left cheek



first_frame[0][28]
first_frame[0][42]
first_frame[0][15]
first_frame[0][14]
first_frame[0][13]
first_frame[0][12]
first_frame[0][54]
#right cheek
'''
#Rectangula actual
left_cheek_x0 = int(first_frame[0][1][0])#x de landmark 2
left_cheek_x1 = int(first_frame[0][31][0])#
left_cheek_y0 = int(first_frame[0][28][1])
left_cheek_y1 = int(first_frame[0][3][1])


right_cheek_x0 = int(first_frame[0][35][0])#x de landmark 2
right_cheek_x1 = int(first_frame[0][15][0])#
right_cheek_y0 = int(first_frame[0][28][1])
right_cheek_y1 = int(first_frame[0][13][1])


face_example = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S014/001/S014_001_00000001.png'
img= cv2.imread(face_example)
#imgleft = img[left_cheek_x0:left_cheek_x1,left_cheek_y0:left_cheek_y1,:]
imgleft = img[left_cheek_y0:left_cheek_y1,left_cheek_x0:left_cheek_x1,:]
imgright = img[right_cheek_y0:right_cheek_y1,right_cheek_x0:right_cheek_x1,:]
#imageRGB = cv2.cvtColor(imgleft, cv2.COLOR_BGR2RGB)

imgleft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)
imgright =cv2.cvtColor(imgright, cv2.COLOR_BGR2GRAY)

cv2.imshow("Face landmark result", imgleft)

# Pause screen to wait key from user to see result
cv2.waitKey(0)
cv2.destroyAllWindows()


In [138]:



subjects_images = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images'
sorted_subjects =   np.sort(os.listdir('../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/')) # we probably need to not have the sorted

first_images = []

for subject in sorted_subjects:
    one_subject_images = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/' + str(subject)+ '/'
    sorted_list = np.sort(os.listdir(one_subject_images)) #creo que este sort se puede quitar
    distances_per_subject = []
    for emotion in sorted_list:
        matrix_landmark_per_frame = []
        subject_landmark_emotion = one_subject_landmarks + str(emotion)                   
        img = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/' +  str(subject)+ '/' + str(emotion)+'/'+ str(subject)+ '_'+str(emotion)+'_'+ str(1).zfill(8)+'.png'            
        first_images.append(img)   
    

In [145]:
import math
first_images = np.asarray(first_images)


nineteen_feature = []
twenty_feature = []
twentyone_feature = []


for i in range(0, len(first_images)):    
    #Rectangula actual
    left_cheek_x0 = int(first_frame[i][1][0])#x de landmark 2
    left_cheek_x1 = int(first_frame[i][31][0])#
    left_cheek_y0 = int(first_frame[i][28][1])
    left_cheek_y1 = int(first_frame[i][3][1])
    
    right_cheek_x0 = int(first_frame[i][35][0])#x de landmark 2
    right_cheek_x1 = int(first_frame[i][15][0])#
    right_cheek_y0 = int(first_frame[i][28][1])
    right_cheek_y1 = int(first_frame[i][13][1])

    img = cv2.imread(first_images[i])
    imgleft = img[left_cheek_y0:left_cheek_y1,left_cheek_x0:left_cheek_x1,:]
    imgright = img[right_cheek_y0:right_cheek_y1,right_cheek_x0:right_cheek_x1,:]
    #imageRGB = cv2.cvtColor(imgleft, cv2.COLOR_BGR2RGB)

    imgleft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)
    imgright =cv2.cvtColor(imgright, cv2.COLOR_BGR2GRAY)
    
    nineteen, twenty, twentyone = cheeks_function(imgleft, imgright, left_cheek_y0,left_cheek_x0,right_cheek_y0,right_cheek_x0)
    nineteen_feature.append(nineteen)
    twenty_feature.append(twenty)
    twentyone_feature.append(twentyone)

In [137]:
      #ESTA ES CON UNA SOLA      

def cheeks_function(imgleft, imgright, left_cheek_y0,left_cheek_x0,right_cheek_y0,right_cheek_x0):
    edgesxleft = cv2.Sobel(imgleft, -1, dx=1, dy=0, ksize=1)
    edgesyleft = cv2.Sobel(imgleft, -1, dx=0, dy=1, ksize=1)

    outputleft = np.zeros((len(edgesxleft),len(edgesxleft[0])))

    for i in range(0, len(edgesxleft)):
     for j in range(0, len(edgesxleft[0])):      
            outputleft[i][j] = math.sqrt(pow(edgesxleft[i][j],2) + pow(edgesyleft[i][j],2)+ pow(-1,2))

    angleleft = np.zeros((len(edgesxleft),len(edgesxleft[0]),3))  
    
    for i in range(0, len(edgesxleft)):
     for j in range(0, len(edgesxleft[0])):
            pos =np.array([edgesxleft[i][j], edgesyleft[i][j], -1]/outputleft[i][j])
            angleleft[i][j] = pos
            
    

    edgesxright = cv2.Sobel(imgright, -1, dx=1, dy=0, ksize=1)
    edgesyright = cv2.Sobel(imgright, -1, dx=0, dy=1, ksize=1)

    outputright = np.zeros((len(edgesxright),len(edgesxright[0])))

    for i in range(0, len(edgesxright)):
     for j in range(0, len(edgesxright[0])):       
            outputright[i][j] = math.sqrt(pow(edgesxright[i][j],2) + pow(edgesyright[i][j],2)+ pow(-1,2))

    angleright = np.zeros((len(edgesxright),len(edgesxright[0]),3))  
    for i in range(0, len(edgesxright)):
     for j in range(0, len(edgesxright[0])):
            pos =np.array([edgesxright[i][j], edgesyright[i][j], -1]/outputright[i][j])
            angleright[i][j] = pos
            
            
    beta = np.array([0,0,1])    



        
    coffleft = np.zeros((len(edgesxleft),len(edgesxleft[0])))
    for i in range(0, len(edgesxleft)):
     for j in range(0, len(edgesxleft[0])):
            coffleft[i][j] = 1- np.dot(angleleft[i][j],beta)

    coffright = np.zeros((len(edgesxright),len(edgesxright[0])))
    for i in range(0, len(edgesxright)):
     for j in range(0, len(edgesxright[0])):
            coffright[i][j] = 1 - np.dot(angleright[i][j],beta)
    
    coff = np.concatenate((coffleft.flatten(), coffright.flatten()))
    coff_sum = coff.sum()
    coff_average = coff.mean()
    
    partial_sumxleft = []
    for i in range(0, len(edgesxleft)):#losy0debencambiarse arriba
     partial_sumxleft.append(coff[i]*(left_cheek_y0+i))

    partial_sumyleft = []
    for i in range(0, len(edgesxleft[0])):#losy0debencambiarse arriba
     partial_sumyleft.append(coff[len(edgesxleft)+i]*(left_cheek_x0+i))
    

    partial_sumxright = []
    for i in range(0, len(edgesxright)):#losy0debencambiarse arriba
     partial_sumxright.append(coff[i+len(edgesxleft)*len(edgesxleft[0])]*(right_cheek_y0+i))

    partial_sumyright = []
    for i in range(0, len(edgesxright[0])):#losy0debencambiarse arriba
     partial_sumyright.append(coff[len(edgesxleft)*len(edgesxleft[0])+len(edgesxright)+i]*(right_cheek_x0+i))

    partial_sumx = np.concatenate((np.asarray(partial_sumxleft),np.asarray(partial_sumxright)))    
    partial_sumy = np.concatenate((np.asarray(partial_sumyleft),np.asarray(partial_sumyright)))    

        

    #HAY QUE PREGUNTAR ESTO
    center_of_massx = np.asarray(partial_sumx).sum()/coff_sum
    center_of_massy = np.asarray(partial_sumy).sum()/coff_sum

      
    return coff_average, center_of_massx, center_of_massy  

        





In [103]:
jo=np.array([[1,2,3],[5,6,7]])
no=np.array([[8,9,10],[11,12,13]])
ve= np.vstack((jo,no))
ve

array([[ 1,  2,  3],
       [ 5,  6,  7],
       [ 8,  9, 10],
       [11, 12, 13]])

In [167]:

structure_vector = first_feature_point
structure_vector = np.vstack((structure_vector, second_feature_point))
structure_vector = np.vstack((structure_vector, third_feature_point))
structure_vector = np.vstack((structure_vector, fourth_feature_point))
structure_vector = np.vstack((structure_vector, five_feature_point))
structure_vector = np.vstack((structure_vector, six_feature_point))
structure_vector = np.vstack((structure_vector, seven_feature_point))
structure_vector = np.vstack((structure_vector, eight_feature_point))
structure_vector = np.vstack((structure_vector, nine_feature_point))
structure_vector = np.vstack((structure_vector, ten_feature_point))
structure_vector = np.vstack((structure_vector, eleven_feature_point))
structure_vector = np.vstack((structure_vector, twelve_feature_point))
structure_vector = np.vstack((structure_vector, thirteen_feature_point))
structure_vector = np.vstack((structure_vector, fourteen_feature_point))
structure_vector = np.vstack((structure_vector, fifteen_feature_point))
structure_vector = np.vstack((structure_vector, sixteen_feature_point))
structure_vector = np.vstack((structure_vector, seventeen_feature_point))
structure_vector = np.vstack((structure_vector, eigtheen_feature_point))
structure_vector = np.vstack((structure_vector, nineteen_feature))
structure_vector = np.vstack((structure_vector, twenty_feature))
structure_vector = np.vstack((structure_vector, twentyone_feature))


In [168]:
structure_vector.shape #cada columna es una instancia

(21, 51)

In [ ]:
#AHORA HAY QUE CONCATENAR LAS EMOCIONES Y PCA 

In [ ]:
#PRIMERO HACEMOS EL CALCULO DE EMOCION

In [176]:
frame_seq_data = pd.read_csv('ckmock_landmark_FramePerSeq.csv',header = None)
frame_seq_data

,0
0,29
1,16
2,30
3,19
4,17
5,30
6,17
7,33
8,23
9,32


In [182]:
frame_seq_array = np.asarray(frame_seq_data)
frame_seq_array.flatten()
cant_frame_per_seq, frame_seq_array.flatten()



(array([29, 16, 30, 19, 17, 30, 17, 33, 23, 32, 17, 19, 12, 29, 25, 14, 24,
        15, 14, 10, 21, 13, 29, 28, 14, 21, 14, 33, 17, 19, 27, 22, 20, 13,
        14, 15, 13, 5, 23, 16, 14, 15, 7, 6, 27, 20, 10, 22, 9, 15, 18],
       dtype=object),
 array([29, 16, 30, 19, 17, 30, 17, 33, 23, 32, 17, 19, 12, 29, 25, 14, 24,
        15, 14, 10, 21, 13, 29, 28, 14, 21, 14, 33, 17, 19, 27, 22, 20, 13,
        14, 15, 13,  5, 23, 16, 14, 15,  7,  6, 27, 20, 10, 22,  9, 15, 18]))

In [170]:
my_data_2 = pd.read_csv('ckmock_landmark_aligned_xy.csv',header = None)
my_data_2

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,-0.163450,-0.156895,-0.145752,-0.136284,-0.122597,-0.098510,-0.067956,-0.033574,0.005142,0.044869,...,0.087766,0.081893,0.069788,0.067553,0.066924,0.067827,0.070162,0.071911,0.072451,0.071782
1,-0.163450,-0.156895,-0.145752,-0.136284,-0.122597,-0.098510,-0.067956,-0.033574,0.005142,0.044869,...,0.087766,0.081893,0.069788,0.067553,0.066924,0.067827,0.070162,0.071911,0.072451,0.071782
2,-0.163450,-0.156895,-0.145752,-0.136284,-0.122597,-0.098510,-0.067956,-0.033574,0.005142,0.044869,...,0.087766,0.081893,0.069788,0.067553,0.066924,0.067827,0.070162,0.071911,0.072451,0.071782
3,-0.163462,-0.156908,-0.145755,-0.136281,-0.122597,-0.098517,-0.067972,-0.033580,0.005137,0.044863,...,0.087772,0.081898,0.069807,0.067551,0.066919,0.067826,0.070184,0.071954,0.072516,0.071826
4,-0.163406,-0.156854,-0.145720,-0.136245,-0.122544,-0.098467,-0.067924,-0.033558,0.005157,0.044874,...,0.087870,0.081942,0.069783,0.067510,0.066874,0.067786,0.070165,0.071976,0.072554,0.071843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,-0.148756,-0.148653,-0.147350,-0.142510,-0.125884,-0.099116,-0.067693,-0.034620,0.002989,0.042856,...,0.087104,0.081242,0.062722,0.061722,0.062258,0.058505,0.057794,0.058807,0.063288,0.062188
961,-0.148750,-0.148671,-0.147348,-0.142510,-0.125914,-0.099120,-0.067702,-0.034628,0.002981,0.042855,...,0.087195,0.081358,0.062807,0.061829,0.062320,0.058664,0.057920,0.058891,0.063298,0.062257
962,-0.148761,-0.148685,-0.147358,-0.142514,-0.125920,-0.099120,-0.067688,-0.034631,0.002974,0.042849,...,0.087149,0.081329,0.062785,0.061839,0.062338,0.058680,0.057926,0.058856,0.063253,0.062230
963,-0.148763,-0.148652,-0.147356,-0.142512,-0.125875,-0.099113,-0.067682,-0.034619,0.002988,0.042853,...,0.087044,0.081181,0.062667,0.061684,0.062242,0.058448,0.057757,0.058756,0.063262,0.062147


In [171]:
#ai deberia ser el maximo de cada columna

Ai = my_data_2.max() # da el maximo por columnas
Ai.shape

(136,)

In [184]:
data_2_intensity = my_data_2
data_2_intensity_div = data_2_intensity/Ai
data_2_intensity_sum =  np.sum(data_2_intensity_div, axis=1)
data_2_intensity_final = data_2_intensity_sum/136
data_2_intensity_final  #the emotion for each frame
#revisar que este bien la cuenta

0      1.037203
1      1.037203
2      1.037203
3      1.037367
4      1.037418
         ...   
960    1.065990
961    1.067019
962    1.066922
963    1.065527
964    1.064312
Length: 965, dtype: float64

In [200]:
sumi=0
cant_features = 22

frame_seq_flattened = frame_seq_array.flatten()

structure_feature_vector = np.zeros((len(data_2_intensity_final), cant_features))
for i in range(0, len(frame_seq_flattened)):
    current_index = frame_seq_flattened[i]
    for j in range(0, current_index):
        vector = structure_vector[:,i]#columna i
        vector = np.append(vector,data_2_intensity_final[j+sumi])        
        structure_feature_vector[j+sumi] = vector
    sumi = sumi + current_index
  

(965, 22)

### AHORA si hacemos PCA con los 22 features

In [201]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
scaled_landDiff = std_scaler.fit_transform(structure_feature_vector)
scaled_landDiff.shape

(965, 22)

In [216]:
from sklearn.decomposition import PCA
pca = PCA(n_components=12)
pca.fit_transform(scaled_landDiff)
b_shape = pca.fit_transform(scaled_landDiff)
b_shape.shape # cada fila deberia ser el b de 1 frame en 12 componentes proncipales (de 22 feature points)

(965, 12)

In [217]:
print(sum(pca.explained_variance_ratio_))


0.9920708753865839


(84, 98)

In [48]:
      #ESTA ES CON UNA SOLA      
import math 

edgesx = cv2.Sobel(imgleft, -1, dx=1, dy=0, ksize=1)
edgesy = cv2.Sobel(imgleft, -1, dx=0, dy=1, ksize=1)



output = np.zeros((len(edgesx),len(edgesx[0])))
for i in range(0, len(edgesx)):
    for j in range(0, len(edgesx[0])):
        output[i][j] = math.sqrt(pow(edgesx[i][j],2) +pow(edgesy[i][j],2)+ pow(-1,2))


angle = np.zeros((len(edgesx),len(edgesx[0]),3))  
for i in range(0, len(edgesx)):
    for j in range(0, len(edgesx[0])):
        pos =np.array([edgesx[i][j], edgesy[i][j], -1]/output[i][j])
        angle[i][j] = pos

beta = np.array([0,0,1])
        
coff = np.zeros((len(edgesx),len(edgesx[0])))
for i in range(0, len(edgesx)):
    for j in range(0, len(edgesx[0])):
        coff[i][j] = 1- np.dot(angle[i][j],beta)

coeff_average = coff.mean()
coeff_sum = coff.sum()

partial_sumx = []
for i in range(0, len(edgesx)):#losy0debencambiarse arriba
    partial_sumx.append(coff[i]*(left_cheek_y0+i))

partial_sumy = []
for i in range(0, len(edgesx[0])):#losy0debencambiarse arriba
    partial_sumy.append(coff[:,i]*(left_cheek_x0+i))
        
center_of_massx = np.asarray(partial_sumx).sum()/coeff_sum
center_of_massy = np.asarray(partial_sumy).sum()/coeff_sum


IndentationError: unindent does not match any outer indentation level (<ipython-input-48-f35130b63526>, line 12)

In [102]:
coff[1,::].shape

(98,)

In [79]:
edgesx[0]

array([ 0,  4,  6,  4,  4,  3,  2,  3,  1,  2,  4,  3,  1,  1,  0,  0,  2,
        4,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  5,  3,  2,  4,  2,  3, 11, 14, 13,
       16, 21, 14,  6,  8,  5,  0,  1,  4,  0,  0,  9, 10,  8,  8,  8,  7,
        7,  8,  8,  5,  2,  3,  3,  1,  0,  0,  2,  3,  0], dtype=uint8)

In [40]:
points = first_frame[0][39:59]
  

rng = np.random.default_rng()
points = rng.random((30, 2))   # 30 random points in 2-D
hull = ConvexHull(points)

20

In [35]:
hull.simplices

array([[14, 15],
       [27, 23],
       [18, 15],
       [18,  6],
       [24,  9],
       [24,  6],
       [ 4, 23],
       [ 4, 14],
       [ 7,  9],
       [ 7, 27]], dtype=int32)

In [38]:
hull.volume

0.807617690185429

In [49]:
#adaptamos los valores de los features a los mas parecidos, el primero al 2 entre cejas porque es la mejor distancia a igual altura y asumo que las distancias se corresponden con el contorno entre facial shapes
#el 4, 14 podria ser 3,13 y tal vez seberia ser la media de ambps
#64,71 media entre las pupilas, ahora estoy midiendo distancia entre las exteriories 37-46, pero cambie a lo mas cercano horizontalmente38-44
feature_horizontal_distance =[]
feature_pairs = np.array([[19, 26], [2, 16], [3,15], [4,14], [7,11], [32,37], [61,65], [38,44],[43,46], [40,43], [37,40]])
for i in range(0,11):
    feature_horizontally = []
    for j in range(0, len(first_frame)):
        point1 = first_frame[j][feature_pairs[i][0]][0]
        point2 = first_frame[j][feature_pairs[i][1]][0]
        feature_horizontally.append(np.abs(point2-point1))
    feature_horizontally = np.asarray( feature_horizontally, dtype=object)
    feature_horizontal_distance.append(feature_horizontally)
feature_horizontal_distance = np.asarray( feature_horizontal_distance, dtype=object)

In [45]:
feature_vertical_distance =[]
feature_pairs = np.array([[19, 26], [2, 16], [3,15], [4,14], [7,11], [32,36], [61,65], [38,44],[43,46], [40,43], [37,40]])
for i in range(0,11):
    feature_vertically = []
    for j in range(0, len(first_frame)):
        point1 = first_frame[j][feature_pairs[i][0]][0]
        point2 = first_frame[j][feature_pairs[i][1]][0]
        feature_vertically.append(np.abs(point2-point1))
    feature_vertically = np.asarray( feature_vertically, dtype=object)
    feature_vertical_distance.append(feature_horizontally)
feature_vertical_distance = np.asarray( feature_vertical_distance, dtype=object)

5

In [ ]:
#Faltan landmarks <-- Hagamos lo de la textura y luego volvemos a esto